In [8]:
import numpy as np
import os
import scipy.linalg as la
from typing import Dict, List, Tuple, Optional  # Fix for NameError 
import rmsynth
from qiskit import QuantumCircuit, transpile
from qiskit.qasm2 import dump
from qiskit.quantum_info import Operator, random_unitary
from qiskit.circuit.library import Diagonal, StatePreparation

In [1]:
import numpy as np
import os
from qiskit import QuantumCircuit, transpile
from qiskit.qasm2 import dump
from qiskit.quantum_info import Statevector, random_unitary
import numpy as np
# ======================================================
# Setup and Synthesis Configuration
# ======================================================
# Strict gate set: H, T, Tdg, and CNOT [cite: 6, 7, 81]
BASIS_GATES = ["h", "t", "tdg", "cx"]
theta = np.pi / 7  # Common rotation angle in the challenge [cite: 25, 28, 30, 33]

def save_challenge_qasm(qc, name):
    """Transpiles and saves the circuit to OpenQASM 2.0 format."""
    optimized_qc = transpile(qc, basis_gates=BASIS_GATES, optimization_level=3)
    filename = f"{name}.qasm"
    dump(optimized_qc, filename)
    print(f"Generated {filename} | T-count: {optimized_qc.count_ops().get('t', 0) + optimized_qc.count_ops().get('tdg', 0)}")


In [3]:
import numpy as np
from qiskit.quantum_info import Operator
from qiskit.qasm2 import dump

def extract_a_coeffs_from_circuit(qc):
    """
    Extracts Z8 phase coefficients (phase = exp(i*pi*k/4)) 
    from a circuit's unitary for rmsynth optimization. [cite: 100]
    """
    U = Operator(qc).data
    # Analyzes the diagonal of the unitary to find phase coefficients [cite: 100]
    phases = np.angle(np.diag(U)) 
    a_coeffs = {i: int(round(4 * p / np.pi)) % 8 for i, p in enumerate(phases) if p != 0}
    return a_coeffs

In [10]:
def save_and_optimize_all(qc, name, n_qubits):
    """
    Optimizes via rmsynth and saves for iQuHACK submission. 
    Only permits {H, T, Tdg, CNOT} gates. [cite: 7, 81]
    """
    try:
        # 1. Coefficient Extraction [cite: 100]
        if name == "11_diagonal":
            # Direct mapping from challenge PDF phases [cite: 55, 60]
            phi_vals = [0, np.pi, 1.25*np.pi, 1.75*np.pi, 1.25*np.pi, 1.75*np.pi, 1.5*np.pi, 1.5*np.pi,
                        1.25*np.pi, 1.75*np.pi, 1.5*np.pi, 1.5*np.pi, 1.5*np.pi, 1.5*np.pi, 1.75*np.pi, 1.25*np.pi]
            a_coeffs = {i: int(round(4 * p / np.pi)) % 8 for i, p in enumerate(phi_vals) if p != 0}
        else:
            # Extract from the unitary matrix for Tasks 1-10 [cite: 100]
            a_coeffs = extract_a_coeffs_from_circuit(qc)

        # 2. Reed-Muller Optimization 
        # This minimizes the number of odd coefficients (T-gates) 
        vec_opt, report, _ = rmsynth.optimize_coefficients(a_coeffs, n_qubits)
        
        # 3. Clifford+T Synthesis [cite: 12, 102]
        # Synthesizes a circuit containing only {H, T, Tdg, CX} [cite: 7, 81]
        optimized_circ = rmsynth.synthesize_from_coeffs(vec_opt, n_qubits)
        
        # 4. Export to OpenQASM [cite: 13, 67]
        filename = f"{name}.qasm"
        with open(filename, "w") as f:
            f.write("OPENQASM 2.0;\ninclude \"qelib1.inc\";\n")
            f.write(f"qreg q[{n_qubits}];\n")
            for op in optimized_circ.ops:
                if op.kind == "cnot": 
                    f.write(f"cx q[{op.ctrl}], q[{op.tgt}];\n")
                elif op.kind == "phase":
                    # Map Z8 exponents to strict gate set [cite: 82]
                    if op.k == 1: f.write(f"t q[{op.q}];\n")
                    elif op.k == 7: f.write(f"tdg q[{op.q}];\n")
                    elif op.k == 4: f.write(f"z q[{op.q}];\n") # Z is Clifford
        
        print(f"✨ {name:18s} | Optimized T-count: {optimized_circ.t_count()}")
        return optimized_circ
    
    except Exception as e:
        # Fallback to standard transpilation if rmsynth fails [cite: 13]
        print(f"⚠️ rmsynth failed for {name}: {e}")
        qc_fallback = transpile(qc, basis_gates=["h", "t", "tdg", "cx"], optimization_level=3)
        dump(qc_fallback, f"{name}.qasm")
    


In [11]:
# ======================================================
# 1. Controlled-Y (Sanity Check) [cite: 18, 24]
# ======================================================
qc1 = QuantumCircuit(2)
qc1.cy(0, 1) 
save_challenge_qasm(qc1, "01_controlled_y")

Generated 01_controlled_y.qasm | T-count: 8


In [12]:
extract_a_coeffs_from_circuit(qc1)  # Test extraction function

{}

In [14]:
circ = save_and_optimize_all(qc1, "01_controlled_y", 2)

✨ 01_controlled_y    | Optimized T-count: 0


In [18]:
# ======================================================
qc2 = QuantumCircuit(2)
qc2.cry(theta, 0, 1)
save_challenge_qasm(qc2, "02_cry_pi_7")
extract_a_coeffs_from_circuit(qc2)  # Test extraction function

Generated 02_cry_pi_7.qasm | T-count: 66438


{}